In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import header
import logging


import sys
import datetime
import pandas as pd
import traceback # 印log
import zipfile #解壓縮檔案

import os
import shutil # 搬檔

In [2]:
# ex:downloadpath = 'D:/國泰世華銀行_法務部'
def downloadFile(finalPath, title, fileUrls, fileNames): # for download pdf or doc
    target = finalPath + '/' + title[:30].strip()
    # 若目錄不存在，建立目錄
    if not os.path.isdir(target):
        os.makedirs(target)
    i = 0
    for file_url in fileUrls:
        try:
            response = requests.get(file_url, stream="TRUE")
            downloadFile = target + '/' + fileNames[i].strip() # 放置資料夾路徑 + 檔名
            print(downloadFile + '\r\n')
            with open(downloadFile,'wb') as file:
                for data in response.iter_content():
                    file.write(data)
        except:
            print("爬取檔案失敗")
            print("失敗連結：" + file_url)
            logging.error("爬取檔案失敗")
            logging.error("失敗連結：" + file_url)
        i += 1

In [13]:
def parsingDetail(df, finalPath): 
#     df2 = pd.DataFrame(columns=['title','link','content','fileNames'])
    df2 = pd.DataFrame(columns = ["標題", "全文內容", "附件", "發文字號", "發文日期", "相關法條"])
    links = df['link']
    titles = df['title']
    dates = df['date']
    fileUrlRoot = 'https://www.banking.gov.tw'
    for i in range(len(titles)):
        title = titles[i]
        print(title)
        link = links[i]
        date = dates[i]
        try:
            soup = request2soup(link)
            content = [e.text for e in soup.select('.page_content')][0]
            str_content = str(soup.select('.page_content')[0])
            fileNames = [e.text for e in soup.select('.acces a')]
            fileUrls = [fileUrlRoot + e.get('href') for e in soup.select('.acces a')]
            serno = re.findall(r'發文字號.+?\d+.+?', str_content)[0][5:]
            print(fileNames)
        except:
            print("爬取內文失敗")
            print("失敗連結：" + link)
            logging.error("爬取內文失敗")
            logging.error("失敗連結：" + link)
            traceback.print_exc()
        if len(fileNames) != 0:
            downloadFile(finalPath, title, fileUrls, fileNames)
        d = {'標題': title, '全文內容': content, '附件':','.join(fileNames), '發文字號':serno, '發文日期':date,
             '相關法條':''}
        df2= df2.append(pd.DataFrame(data=d, index=[0]))
    return df2

In [4]:
def outputCsv(df, fileName, path):
    # 若目錄不存在，建立目錄
    if not os.path.isdir(path):
        os.mkdir(path)
    df.to_csv(path + "/" + fileName + ".csv", index = False, encoding = "utf_8_sig")

In [5]:
def compareTo(strDate, endDate):
    if int(re.split(r'(/|-|\.)', strDate)[0]) < 1911:
        strDate = datetime.datetime.strptime(str(int(re.sub(r'(/|-|\.)', '', strDate)) + 19110000), "%Y%m%d").strftime("%Y-%m-%d")
    if int(re.split(r'(/|-|\.)', endDate)[0]) < 1911:
        endDate = datetime.datetime.strptime(str(int(re.sub(r'(/|-|\.)', '', endDate)) + 19110000), "%Y%m%d").strftime("%Y-%m-%d")
    try:
        strDate = datetime.datetime.strptime(strDate, "%Y-%m-%d")
        endDate = datetime.datetime.strptime(endDate, "%Y-%m-%d")

    except:
        print("日期格式錯誤：strDate = %s, endDate = %s" %(strDate, endDate))
        logging.error('compareTo(strDate, endDate):')
        logging.error("日期格式錯誤：strDate = %s, endDate = %s" %(strDate, endDate))
        return
    if strDate < endDate:
        return 1
    elif strDate == endDate:
        return 0
    else:
        return -1


In [6]:
def parsingTitle(soup, checkRange):
    
    try:
        # 取得上次爬網結果
        lastResultPath = "./CrawlList/lastResult.csv"
        if os.path.isfile(lastResultPath):
            lastResult = pd.read_csv(lastResultPath)
        else:
            lastResult = pd.DataFrame()
        
        # 爬網日期區間為一個禮拜
        endDate = datetime.date.today()
        strDate = (endDate - datetime.timedelta(days = checkRange)).isoformat()
        df = pd.DataFrame(columns = ["date", "title", "link"])
        nowPage = 1
        preurl = 'https://www.banking.gov.tw/ch/'
        titles_result = []
        dates = []
        links = []
        while True:
            
            try:
                if (nowPage > 1):
                    soup = request2soup(url, nowPage)
                titles = [str(e.get('title').strip()) for e in soup.select('.ptitle1 a')]
                if titles == []:
                    break
                for index in range(len(titles)):
                    title = titles[index]
                    date = soup.select('.pdate1')[index].text.strip()
                    if compareTo(date, strDate) > 0: # 若發文日期小於開始日期, 則結束爬取主旨
                        break
                    link = preurl + soup.select('.ptitle1 a')[index].get('href')
                    titles_result.append(title)
                    dates.append(date)
                    links.append(link)
                nowPage += 1
            except:
                print("爬取第 %s 頁第 %s 筆主旨發生錯誤" %(nowPage, index + 1))
                logging.error("爬取第 %s 頁第 %s 筆主旨發生錯誤" %(nowPage, index + 1))
                traceback.print_exc()
        
        d = {'date': dates, 'title': titles_result, 'link': links}
        df = df.append(pd.DataFrame(data=d))    
        
        if not lastResult.empty:
            # 若與上次發文日期和標題相同，則跳至下一筆
            for i in range(len(df)):
                for j in range(len(lastResult)):
                    if (df.date[i] == lastResult.date[j]) & (df.title[i] == lastResult.title[j]): 
                        df.drop(i, inplace = True)
                        break

        if len(df) == 0:
            print("%s 至 %s 間無資料更新" %(strDate, endDate))
            logging.critical("%s 至 %s 間無資料更新" %(strDate, endDate))
        else:
            df.index = [i for i in range(df.shape[0])] # reset
            outputCsv(df, "lastResult", "./CrawlList")
        
        return df
    except:
        print("爬取主旨列表失敗")
        logging.error("爬取主旨列表失敗")
        traceback.print_exc()
    
    


In [7]:
def request2soup(url, page = None):
    if page is None:
        res = requests.get(url)
    else:
        formData = {'id':'190',
                'contentid':'190',
                'parentpath':'0,3',
                'mcustomize':'lawnew_list.jsp',
                'keyword':'請輸入查詢關鍵字',           
                'page':page} 
        res = requests.post(url, data = formData)
        
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser", from_encoding = "utf-8")
    return soup

In [8]:
def main(url, checkRange = 18):
    
    logging.critical("\n")
    logging.critical("爬網開始......")
    logging.critical("目標網址：" + url)
    
    strTime = datetime.datetime.now()
    logging.critical("開始時間：" + strTime.strftime("%Y/%m/%d %H:%M:%S"))
    workPath = os.getcwd()
    TempPath = "./Temp"  # browser file
    FinalPath = "./Result" # project file
    
    try:
        soup = request2soup(url, 1)
        df_1 = parsingTitle(soup, checkRange)
        if len(df_1) == 0:
            return
        outputCsv(df_1, "第一層結果", FinalPath)

        df_2 = parsingDetail(df_1, FinalPath)
        outputCsv(df_2, "第二層結果", FinalPath)
    except:
        print("執行爬網作業失敗")
        logging.error("執行爬網作業失敗")
        traceback.print_exc()
        
    endTime = datetime.datetime.now()
    logging.critical("結束時間：" + endTime.strftime("%Y/%m/%d %H:%M:%S"))
    logging.critical("執行時間：" + str((endTime - strTime).seconds) + " 秒")
    logging.critical("輸出筆數：" + str(len(df_1)) + " 筆")
    logging.critical("爬網結束......")

In [15]:
if __name__ == "__main__":
    url = "https://www.banking.gov.tw/ch/home.jsp?id=190&parentpath=0,3"
    today = datetime.datetime.today().strftime("%Y-%m-%d")
    main(url)

C:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:179: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


修正「金融機構對經指定制裁對象之財物或財產上利益及所在地通報辦法」第三條
['10702745350修正總說明及條文對照表.pdf', '10702745350通報辦法.pdf']
./Result/修正「金融機構對經指定制裁對象之財物或財產上利益及所在地通報/10702745350修正總說明及條文對照表.pdf

./Result/修正「金融機構對經指定制裁對象之財物或財產上利益及所在地通報/10702745350通報辦法.pdf

修正「金融機構防制洗錢辦法」。
['1070274522金融機構防制洗錢辦法條文.pdf', '1070274522修正總說明及條文對照表.pdf']
./Result/修正「金融機構防制洗錢辦法」。/1070274522金融機構防制洗錢辦法條文.pdf

./Result/修正「金融機構防制洗錢辦法」。/1070274522修正總說明及條文對照表.pdf

修正「商業銀行設立標準」部分條文及「商業銀行轉投資應遵守事項準則」
['10702745620設立標準總說明及條文對照表.pdf', '10702745620商銀轉投資準則.pdf', '10702745620轉投資總說明及條文對照表.pdf', '10702745620商銀設立標準部分修正條文(發布).pdf']
./Result/修正「商業銀行設立標準」部分條文及「商業銀行轉投資應遵守事項/10702745620設立標準總說明及條文對照表.pdf

./Result/修正「商業銀行設立標準」部分條文及「商業銀行轉投資應遵守事項/10702745620商銀轉投資準則.pdf

./Result/修正「商業銀行設立標準」部分條文及「商業銀行轉投資應遵守事項/10702745620轉投資總說明及條文對照表.pdf

./Result/修正「商業銀行設立標準」部分條文及「商業銀行轉投資應遵守事項/10702745620商銀設立標準部分修正條文(發布).pdf

公告申請設立純網路銀行有關事項
['10702745626附件.zip']
./Result/公告申請設立純網路銀行有關事項/10702745626附件.zip

公告申請設立純網路銀行問答集
[]
訂定「辦理融資性租賃業務事業防制洗錢及打擊資恐內部控制與稽核制度實施辦法」
['10702744580融資條文.pdf', '1